In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


# URL del sitio web de Claro Paraguay
url_tienda_claro = "https://tienda.claro.com.py/"

- **Instanciar** un driver del navegador de preferencia.

In [2]:
options = webdriver.EdgeOptions()
options.add_argument(' --force-device-scale-factor=0.8')
options.add_argument(' --incognito')
driver = webdriver.Edge(service=Service(executable_path=EdgeChromiumDriverManager().install()), options=options)

- Pasarla url de la pagina de preferencia

In [3]:
driver.get(url_tienda_claro)

- Boton para seguir a la pagina siguiente

In [13]:
siguiente_pagina = driver.find_element(By.XPATH, '//*[@id="tgPaginadorTop"]/div/a[4]/i')
siguiente_pagina.click()

- Mapeo de los campos de las caracteristicas y creacion del diccionario para almacenear las caracteristicas de los dispositivos.

In [82]:
attribute_mapping = {
    "Marca": 'brand',
    "Memoria Interna": 'internal_memory',
    "Cámara": 'camera',
    "Batería": 'battery',
    "Sistema Operativo": 'operating_system',
    "Display": 'display_size',
}

# Características
feature_dic = { 'device': [], 'price': []}
feature_dic.update({key: [] for key in attribute_mapping.values()})


- Creacion de la funcion para raspar las caracteristica de cada dispostivo en una pagina.

In [79]:
def extrear_caracteristicas_pagina_1():

    # Obtener dispositivos
    lista_dispositivos = driver.find_elements(By.XPATH, '//div[@id="tgEquipos"]/child::*')

    for _ in range(len(lista_dispositivos)):
        # Esperar a que la página se cargue completamente
        driver.implicitly_wait(10)

        # Filtrar solo los SmartPhones
        driver.find_element(By.XPATH, '//*[@id="5"]').click()

        # Esperar a que los resultados se carguen
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="tgEquipos"]/child::*')))

        # Obtener la lista de dispositivos nuevamente
        lista_dispositivos = driver.find_elements(By.XPATH, '//div[@id="tgEquipos"]/child::*')

        # Entrar al detalle del dispositivo
        dispositivo = lista_dispositivos[_].find_element(By.XPATH, './/a[contains(@onclick, "getArrayEquipo(")]')
        dispositivo.click()

        # Extraer Nombre y Precio
        device = driver.find_element(By.XPATH, '//*[@id="menu-01"]/div[1]/div/div/div[2]/h2').text
        price = driver.find_element(By.XPATH, '//*[@id="dv_MainContainerEquiposResumen"]/div[2]/div/div[2]/span[2]').text

        feature_dic['device'].append(device)
        feature_dic['price'].append(price)

        # Extraer atributos adicionales del dispositivo
        attributes = driver.find_elements(By.XPATH, '//div[@id="menu-01"]/div[2]/div/div')
        
        # lista de caracteristicas del mapeo que aparecieron
        aparecidos_name = []
        for attribute in attributes:
            name_element = attribute.find_element(By.XPATH, './/h3')
            value_element = attribute.find_element(By.XPATH, './/p')

            name = name_element.text.strip()
            value = value_element.text.strip() if value_element.text else None
            aparecidos_name.append(name)

            # Verificar si el nombre del atributo está en el mapeo
            if name in attribute_mapping:
                field_name = attribute_mapping[name]
                feature_dic[field_name].append(value)
        
        # Rellenar con "null" los atributos no aparecidos
        for key, value in attribute_mapping.items():
            if key not in aparecidos_name:
                feature_dic[value].append("null")

        # Volver a la página principal
        pagina_principal = driver.find_element(By.XPATH, '//a[@href="https://tienda.claro.com.py"]')
        pagina_principal.click()


def extrear_caracteristicas_pagina_2():

    # Filtrar solo los SmartPhones y cambiar de pagina
    driver.find_element(By.XPATH, '//*[@id="5"]').click()
    driver.find_element(By.XPATH, '//*[@id="tgPaginadorTop"]/div/a[4]/i').click()

    # Obtener dispositivos
    lista_dispositivos = driver.find_elements(By.XPATH, '//div[@id="tgEquipos"]/child::*')

    for _ in range(len(lista_dispositivos)):
        # Esperar a que la página se cargue completamente
        driver.implicitly_wait(10)

        # Filtrar solo los SmartPhones y cambiar de pagina
        # driver.find_element(By.XPATH, '//*[@id="5"]').click()
        # driver.find_element(By.XPATH, '//*[@id="tgPaginadorTop"]/div/a[4]/i').click()

        # Esperar a que los resultados se carguen
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="tgEquipos"]/child::*')))

        # Obtener la lista de dispositivos nuevamente
        lista_dispositivos = driver.find_elements(By.XPATH, '//div[@id="tgEquipos"]/child::*')

        # Entrar al detalle del dispositivo
        lista_dispositivos[_].find_element(By.XPATH, './/a[contains(@onclick, "getArrayEquipo(")]').click()
     
        # Extraer Nombre y Precio
        device = driver.find_element(By.XPATH, '//*[@id="menu-01"]/div[1]/div/div/div[2]/h2').text
        price = driver.find_element(By.XPATH, '//*[@id="dv_MainContainerEquiposResumen"]/div[2]/div/div[2]/span[2]').text

        feature_dic['device'].append(device)
        feature_dic['price'].append(price)

        # Extraer atributos adicionales del dispositivo
        attributes = driver.find_elements(By.XPATH, '//div[@id="menu-01"]/div[2]/div/div')
        
        # lista de caracteristicas del mapeo que aparecieron
        aparecidos_name = []
        for attribute in attributes:
            name_element = attribute.find_element(By.XPATH, './/h3')
            value_element = attribute.find_element(By.XPATH, './/p')

            name = name_element.text.strip()
            value = value_element.text.strip() if value_element.text else None
            aparecidos_name.append(name)

            # Verificar si el nombre del atributo está en el mapeo
            if name in attribute_mapping:
                field_name = attribute_mapping[name]
                feature_dic[field_name].append(value)
        
        # Rellenar con "null" los atributos no aparecidos
        for key, value in attribute_mapping.items():
            if key not in aparecidos_name:
                feature_dic[value].append("null")

        # Volver a la página principal
        driver.find_element(By.XPATH, '//a[@href="https://tienda.claro.com.py"]').click()



In [85]:
extrear_caracteristicas_pagina_1()

In [86]:
extrear_caracteristicas_pagina_2()

- Crear un objecto de tipo DataFrame para almacenar los datos del diccionario.

In [87]:
df = pd.DataFrame(feature_dic)
df

,device,price,brand,internal_memory,camera,battery,operating_system,display_size
0,Samsung Galaxy A04 64gb,Gs.1.530.000,Samsung,null,50 + 2 MP / 5 MP,5000 mAh,Android 12,6.5''
1,Nokia C2 2nd Edition,Gs.907.200,Nokia,ROM: 32 GB | RAM: 2 GB,5 MP / 2 MP,2400 mAh,Android GO 11,5.7'' FWVGA+
2,Samsung Galaxy Z Flip5,Gs.9.900.000,Samsung,ROM: 256 GB | RAM: 8 GB,12 + 12 MP / 10 MP,3700 mAh,Android 13,6.7''
3,ZTE Blade A71,Gs.722.925,Marca ZTE,ROM: 64 GB | RAM: 3 GB,16 + 8 + 2 MP / 8 MP,4000 mAh,Android 11,6.5''
4,Moto E13,Gs.842.400,Motorola,ROM: 64 GB | RAM: 2 GB,13 MP / 5 MP,5000 mAh,Android 11,6.5''
5,Motorola G13,Gs.1.620.000,Motorola,ROM: 128 GB | RAM: 4 GB,50 + 2 + 2 MP / 8 MP,5000 mAh,Android 11,6.5''
6,ZTE Blade A31 Lite,Gs.344.250,Marca ZTE,ROM: 32 GB | RAM: 1 GB,8 MP / 5 MP,2000 mAh,Android 9.0,5.0''
7,ZTE Blade A33s,Gs.702.000,Marca ZTE,ROM: 32 GB | RAM: 2 GB,5 MP / 2 MP,3000 mAh,Android 12,6.26''
8,Samsung Galaxy A04e 64gb,Gs.1.206.000,Samsung,ROM: 64 GB | RAM: 3 GB,13 + 2 MP / 5 MP,5000 mAh,Android 12,6.5''
9,Samsung Galaxy A24,Gs.2.880.000,Samsung,ROM: 128 GB | RAM: 4 GB,50 + 5 + 2 MP / 13 MP,5000 mAh,Android 13,6.5''


- Exporta el DataFrame en un formato `.csv`

In [ ]:
df.to_csv('Tienda_Claro_Paraguay.csv')

- Terminar la ejecucion de la instancia del driver

In [ ]:
driver.close()